# School Census Data Extraction

This notebook goal is to create an school dataset with geographical location, census data and education indicators.


In [1]:
import numpy as np
import pandas as pd
import geobr

Usually the main place where we can find useful data is the Official Government sources. In this case, we will download our dataset from
gov.br.

**Data Source**

- Ministry: Ministério da Educação
- Entity: Instituto Nacional de Estudos e Pesquisas Educacionais Anísio Teixeira | Inep
- Data: Microdados do Censo Escolar da Educação Básica 2022
- [Link](https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/censo-escolar)


In [2]:
censo_edu_2022 = pd.read_csv(
    "data/microdados_ed_basica_2022.csv", sep=";", encoding="latin-1"
)

/var/folders/2v/rfkdf8zd7yx_k7lff_2bchrr0000gn/T/ipykernel_8678/1192548722.py:1: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  censo_edu_2022 = pd.read_csv(


In [3]:
# Check the first 5 rows
censo_edu_2022.head()

,NU_ANO_CENSO,NO_REGIAO,CO_REGIAO,NO_UF,SG_UF,CO_UF,NO_MUNICIPIO,CO_MUNICIPIO,NO_MESORREGIAO,CO_MESORREGIAO,...,QT_TUR_FUND_AF,QT_TUR_MED,QT_TUR_PROF,QT_TUR_PROF_TEC,QT_TUR_EJA,QT_TUR_EJA_FUND,QT_TUR_EJA_MED,QT_TUR_ESP,QT_TUR_ESP_CC,QT_TUR_ESP_CE
0,2022,Norte,1,Rondônia,RO,11,Alta Floresta D'Oeste,1100015,Leste Rondoniense,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2022,Norte,1,Rondônia,RO,11,Alta Floresta D'Oeste,1100015,Leste Rondoniense,2,...,0.0,0.0,0.0,0.0,10.0,6.0,4.0,3.0,3.0,0.0
2,2022,Norte,1,Rondônia,RO,11,Alta Floresta D'Oeste,1100015,Leste Rondoniense,2,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2022,Norte,1,Rondônia,RO,11,Alta Floresta D'Oeste,1100015,Leste Rondoniense,2,...,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,2022,Norte,1,Rondônia,RO,11,Alta Floresta D'Oeste,1100015,Leste Rondoniense,2,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0


In [4]:
# Check the shape (# rows, # columns) of the dataframe
censo_edu_2022.shape

(224649, 385)

After reviewing the data diccionary that comes with the dataset, we will extract the following variables:


In [5]:
selected_cols = [
    # Identification
    "NO_ENTIDADE",
    "CO_ENTIDADE",
    # Administrative information
    "TP_DEPENDENCIA",
    "TP_CATEGORIA_ESCOLA_PRIVADA",
    "TP_LOCALIZACAO",
    "TP_LOCALIZACAO_DIFERENCIADA",
    # Classrooms utilization
    "QT_SALAS_UTILIZADAS_DENTRO",
    "QT_SALAS_UTILIZADAS_FORA",
    "QT_SALAS_UTILIZADAS",
    # Students enrollment
    ######## NEW VARIABLES ########
    "QT_MAT_BAS_D",  # Número de Matrículas na Educação Básica - Turno Diurno
    "QT_MAT_BAS_N",  # Número de Matrículas na Educação Básica - Turno Noturno
    "QT_MAT_BAS_EAD",  # Número de Matrículas na Educação Básica - Turmas semipresenciais ou de Educação a Distância (EAD)
    "QT_MAT_INF_INT",  # Número de Matrículas na Educação Infantil - Tempo Integral
    "QT_MAT_INF_CRE_INT",  # Número de Matrículas na Educação Infantil - Creche - Tempo Integral
    "QT_MAT_INF_PRE_INT",  # Número de Matrículas na Educação Infantil - Pré-Escola - Tempo Integral
    "QT_MAT_FUND_INT",  # Número de Matrículas no Ensino Fundamental - Tempo Integral
    "QT_MAT_FUND_AI_INT",  # Número de Matrículas no Ensino Fundamental - Anos Iniciais - Tempo Integral
    "QT_MAT_FUND_AF_INT",  # Número de Matrículas no Ensino Fundamental - Anos Finais - Tempo Integral
    "QT_MAT_MED_INT",  # Número de Matrículas no Ensino Médio - Tempo Integral
    ######## NEW VARIABLES ########
    "QT_MAT_BAS",
    "QT_MAT_INF",
    "QT_MAT_INF_CRE",
    "QT_MAT_INF_PRE",
    "QT_MAT_FUND",
    "QT_MAT_FUND_AI",
    "QT_MAT_FUND_AF",
    "QT_MAT_MED",
    "QT_MAT_PROF",
    "QT_MAT_PROF_TEC",
    "QT_MAT_EJA",
    "QT_MAT_EJA_FUND",
    "QT_MAT_EJA_MED",
    "QT_MAT_ESP",
    "QT_MAT_ESP_CC",
    "QT_MAT_ESP_CE",
    "QT_MAT_BAS_0_3",
    "QT_MAT_BAS_4_5",
    "QT_MAT_BAS_6_10",
    "QT_MAT_BAS_11_14",
    "QT_MAT_BAS_15_17",
    "QT_MAT_BAS_18_MAIS",
    # Teachers
    "QT_DOC_BAS",
    "QT_DOC_INF",
    "QT_DOC_INF_CRE",
    "QT_DOC_INF_PRE",
    "QT_DOC_FUND",
    "QT_DOC_FUND_AI",
    "QT_DOC_FUND_AF",
    "QT_DOC_MED",
    "QT_DOC_PROF",
    "QT_DOC_PROF_TEC",
    "QT_DOC_EJA",
    "QT_DOC_EJA_FUND",
    "QT_DOC_EJA_MED",
    "QT_DOC_ESP",
    "QT_DOC_ESP_CC",
    "QT_DOC_ESP_CE",
    # Class (Group of students)
    "QT_TUR_BAS",
    "QT_TUR_INF",
    "QT_TUR_INF_CRE",
    "QT_TUR_INF_PRE",
    "QT_TUR_FUND",
    "QT_TUR_FUND_AI",
    "QT_TUR_FUND_AF",
    "QT_TUR_MED",
    "QT_TUR_PROF",
    "QT_TUR_PROF_TEC",
    "QT_TUR_EJA",
    "QT_TUR_EJA_FUND",
    "QT_TUR_EJA_MED",
    "QT_TUR_ESP",
    "QT_TUR_ESP_CE",
    "QT_TUR_ESP_CC",
]

In [6]:
len(selected_cols)

73

Before selecting our variables and saving our final dataset in a new file. We still need to include the geographical location of each school.

For this, we will use the geobr package $^1$ to get the latitude and longitude of each school.

$^1$ Pereira R, Goncalves C (2024). geobr: Download Official Spatial Data Sets of Brazil. R package version 1.8.2, https://github.com/ipeaGIT/geobr, https://ipeagit.github.io/geobr/.


In [7]:
brazil_schools = geobr.read_schools(year=2020)  # 2020 is the most recent year available

In [8]:
# Check the first 5 rows
brazil_schools.head()

,abbrev_state,name_muni,code_school,name_school,education_level,education_level_others,admin_category,address,phone_number,government_level,private_school_type,private_government_partnership,regulated_education_council,service_restriction,size,urban,location_type,date_update,geometry
0,RO,Porto Velho,11000023,EEEE ABNAEL MACHADO DE LIMA - CENE,Ensino Fundamental,Atendimento Educacional Especializado,Pública,"AVENIDA AMAZONAS, 6492 ZONA LESTE. TIRADENTES....",(69) 992083054,Estadual,Não Informado,Não,Não,ESCOLA ATENDE EXCLUSIVAMENTE ALUNOS COM DEFICI...,Entre 51 e 200 matrículas de escolarização,Urbana,A escola não está em área de localização difer...,2020-10-18,POINT (-63.85401 -8.75846)
1,RO,Porto Velho,11000040,EMEIEF PEQUENOS TALENTOS,Educação Infantil,,Pública,"RUA CAETANO, 3256 PREDIO. CALADINHO. 76808-108...",(69) 32135237,Municipal,Não Informado,Não,Sim,ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATE...,Entre 201 e 500 matrículas de escolarização,Urbana,A escola não está em área de localização difer...,2020-10-18,POINT (-63.88392 -8.79373)
2,RO,Porto Velho,11000058,CENTRO DE ENSINO CLASSE A,"Educação Infantil, Ensino Fundamental, Ensino ...",,Privada,"AVENIDA CARLOS GOMES, 1135 CENTRO. 76801-123 P...",(69) 32244473,Privada,Particular,Não,Sim,ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATE...,Mais de 1000 matrículas de escolarização,Urbana,A escola não está em área de localização difer...,2020-10-18,POINT (-63.90199 -8.76073)
3,RO,Porto Velho,11000082,CENTRO EDUCACIONAL PRESBITERIANO 15 DE NOVEMBRO,"Educação Infantil, Ensino Fundamental",,Privada,"RUA ALMIRANTE BARROSO, 1483 SANTA BARBARA. 768...",(69) 32245636,Privada,Particular,Não,Sim,ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATE...,Entre 51 e 200 matrículas de escolarização,Urbana,A escola não está em área de localização difer...,2020-10-18,POINT (-63.89618 -8.76520)
4,RO,Porto Velho,11000104,CENTRO EDUC CORA CORALINA,"Educação Infantil, Ensino Fundamental",,Privada,"RUA MEXICO, 1056 NOVA PORTO VELHO. 76820-190 P...",(69) 32252616,Privada,Particular,Não,Sim,ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATE...,Entre 501 e 1000 matrículas de escolarização,Urbana,A escola não está em área de localização difer...,2020-10-18,POINT (-63.87547 -8.76863)


In [9]:
# Check the shape (# rows, # columns) of the dataframe
brazil_schools.shape

(222936, 19)

Let's compare the number of schools in the two datasets.


In [10]:
print("Number of schools in Microdata:", censo_edu_2022.shape[0])
print("Number of schools in Geobr:", brazil_schools.shape[0])

print(
    "Difference (GeoBr - Microdata):",
    diff := brazil_schools.shape[0] - censo_edu_2022.shape[0],
    f"({diff / brazil_schools.shape[0]:.4f}%)",
)

Number of schools in Microdata: 224649
Number of schools in Geobr: 222936
Difference (GeoBr - Microdata): -1713 (-0.0077%)


Since our project needs to use the location data, we will use the geobr dataset as the main dataset.

Then we will add the school variables to the corresponding location using the school code as the key.


In [11]:
brazil_schools_census_data = brazil_schools.merge(
    right=censo_edu_2022[selected_cols],
    how="left",  # This indicates that brazil_schools is the main dataframe
    left_on="code_school",
    right_on="CO_ENTIDADE",
)

In [12]:
# Check the first 5 rows
brazil_schools_census_data.head()

,abbrev_state,name_muni,code_school,name_school,education_level,education_level_others,admin_category,address,phone_number,government_level,...,QT_TUR_FUND_AF,QT_TUR_MED,QT_TUR_PROF,QT_TUR_PROF_TEC,QT_TUR_EJA,QT_TUR_EJA_FUND,QT_TUR_EJA_MED,QT_TUR_ESP,QT_TUR_ESP_CE,QT_TUR_ESP_CC
0,RO,Porto Velho,11000023,EEEE ABNAEL MACHADO DE LIMA - CENE,Ensino Fundamental,Atendimento Educacional Especializado,Pública,"AVENIDA AMAZONAS, 6492 ZONA LESTE. TIRADENTES....",(69) 992083054,Estadual,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,13.0,0.0
1,RO,Porto Velho,11000040,EMEIEF PEQUENOS TALENTOS,Educação Infantil,,Pública,"RUA CAETANO, 3256 PREDIO. CALADINHO. 76808-108...",(69) 32135237,Municipal,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,11.0
2,RO,Porto Velho,11000058,CENTRO DE ENSINO CLASSE A,"Educação Infantil, Ensino Fundamental, Ensino ...",,Privada,"AVENIDA CARLOS GOMES, 1135 CENTRO. 76801-123 P...",(69) 32244473,Privada,...,14.0,8.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0
3,RO,Porto Velho,11000082,CENTRO EDUCACIONAL PRESBITERIANO 15 DE NOVEMBRO,"Educação Infantil, Ensino Fundamental",,Privada,"RUA ALMIRANTE BARROSO, 1483 SANTA BARBARA. 768...",(69) 32245636,Privada,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0
4,RO,Porto Velho,11000104,CENTRO EDUC CORA CORALINA,"Educação Infantil, Ensino Fundamental",,Privada,"RUA MEXICO, 1056 NOVA PORTO VELHO. 76820-190 P...",(69) 32252616,Privada,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,11.0


In [13]:
# Check the shape (# rows, # columns) of the dataframe
brazil_schools_census_data.shape

(222936, 92)

In [14]:
# Verify the percentage of missing values for the quantitative columns
brazil_schools_census_data[
    [col for col in brazil_schools_census_data.columns if "QT_" in col]
].isna().sum().sort_values(ascending=False) / len(brazil_schools_census_data)

QT_DOC_BAS                    0.234188
QT_DOC_ESP_CE                 0.234188
QT_DOC_INF                    0.234188
QT_DOC_INF_CRE                0.234188
QT_DOC_INF_PRE                0.234188
                                ...   
QT_MAT_BAS_6_10               0.233278
QT_MAT_BAS_D                  0.233278
QT_SALAS_UTILIZADAS           0.226078
QT_SALAS_UTILIZADAS_FORA      0.226078
QT_SALAS_UTILIZADAS_DENTRO    0.226078
Length: 67, dtype: float64

In [15]:
# Calculate ratio of students per teacher for each education level. Example:  QT_MAT_BAS / QT_DOC_BAS, QT_MAT_INF / QT_DOC_INF, etc.
edu_levels = [
    "BAS",
    "INF",
    "INF_CRE",
    "INF_PRE",
    "FUND",
    "FUND_AI",
    "FUND_AF",
    "MED",
    "PROF",
    "PROF_TEC",
    "EJA",
    "EJA_FUND",
    "EJA_MED",
    "ESP",
    "ESP_CC",
    "ESP_CE",
]
for edu_level in edu_levels:
    brazil_schools_census_data[f"ratio_MAT_DOC_{edu_level}"] = (
        brazil_schools_census_data[f"QT_MAT_{edu_level}"]
        / brazil_schools_census_data[f"QT_DOC_{edu_level}"]
    )

In [16]:
# Get descriptive statistics for the ratio of students per teacher for each education level
# Rows that contain missing values are dropped
brazil_schools_census_data[
    [col for col in brazil_schools_census_data.columns if "ratio_MAT_DOC" in col]
].dropna(how="all").describe().round()

/Users/claudio/Documents/edu-brazil/.env/lib/python3.11/site-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/claudio/Documents/edu-brazil/.env/lib/python3.11/site-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/claudio/Documents/edu-brazil/.env/lib/python3.11/site-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/claudio/Documents/edu-brazil/.env/lib/python3.11/site-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/claudio/Documents/edu-brazil/.env/lib/python3.11/site-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/claudio/

,ratio_MAT_DOC_BAS,ratio_MAT_DOC_INF,ratio_MAT_DOC_INF_CRE,ratio_MAT_DOC_INF_PRE,ratio_MAT_DOC_FUND,ratio_MAT_DOC_FUND_AI,ratio_MAT_DOC_FUND_AF,ratio_MAT_DOC_MED,ratio_MAT_DOC_PROF,ratio_MAT_DOC_PROF_TEC,ratio_MAT_DOC_EJA,ratio_MAT_DOC_EJA_FUND,ratio_MAT_DOC_EJA_MED,ratio_MAT_DOC_ESP,ratio_MAT_DOC_ESP_CC,ratio_MAT_DOC_ESP_CE
count,170727.0,107560.0,69217.0,95432.0,119282.0,102457.0,78816.0,28775.0,8210.0,7509.0,29787.0,24591.0,9200.0,128215.0,126499.0,3049.0
mean,16.0,inf,inf,inf,inf,inf,inf,inf,inf,inf,14.0,14.0,11.0,inf,inf,inf
std,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0,11.0,62.0,NaN,NaN,NaN
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,10.0,9.0,8.0,9.0,9.0,11.0,2.0,6.0,5.0,5.0,6.0,6.0,4.0,0.0,0.0,3.0
50%,15.0,14.0,12.0,14.0,13.0,17.0,8.0,9.0,9.0,9.0,11.0,12.0,6.0,1.0,1.0,5.0
75%,20.0,20.0,21.0,21.0,18.0,25.0,14.0,15.0,18.0,18.0,18.0,19.0,11.0,1.0,1.0,8.0
max,8950.0,inf,inf,inf,inf,inf,inf,inf,inf,inf,4104.0,346.0,4014.0,inf,inf,inf


In [17]:
# Calculate ratio of students per classroom in general
edu_levels = ["BAS", "INF", "FUND", "MED", "PROF", "PROF_TEC", "EJA", "ESP"]
brazil_schools_census_data["ratio_MAT_SALAS"] = (
    brazil_schools_census_data[[f"QT_MAT_{edu_level}" for edu_level in edu_levels]].sum(
        axis=1
    )
    / brazil_schools_census_data["QT_SALAS_UTILIZADAS"]
)

In [18]:
len(
    [
        "abbrev_state",
        "name_muni",
        "code_school",
        "name_school",
        "education_level",
        "education_level_others",
        "admin_category",
        "address",
        "phone_number",
        "government_level",
        "private_school_type",
        "private_government_partnership",
        "regulated_education_council",
        "service_restriction",
        "size",
        "urban",
        "location_type",
        "date_update",
        "geometry",
    ]
)

19

The first 19 columns corresponding to variables previously present on the schools geospatial dataset. All others are the newly added variables.


In [19]:
new_vars_na_count = (
    brazil_schools_census_data[brazil_schools_census_data.columns[19:]]
    .isna()
    .sum()
    .sort_values(ascending=False)
)
new_vars_na_prop = new_vars_na_count / len(brazil_schools_census_data)

for i in range(0, len(new_vars_na_count)):
    print(
        f"{new_vars_na_count.index[i]}: {new_vars_na_count[i]} ({new_vars_na_prop[i]:.2%})"
    )

ratio_MAT_DOC_ESP_CE: 219887 (98.63%)
ratio_MAT_DOC_PROF_TEC: 215427 (96.63%)
ratio_MAT_DOC_PROF: 214726 (96.32%)
ratio_MAT_DOC_EJA_MED: 213736 (95.87%)
ratio_MAT_DOC_EJA_FUND: 198345 (88.97%)
ratio_MAT_DOC_MED: 194161 (87.09%)
ratio_MAT_DOC_EJA: 193149 (86.64%)
TP_CATEGORIA_ESCOLA_PRIVADA: 185958 (83.41%)
ratio_MAT_DOC_INF_CRE: 153719 (68.95%)
ratio_MAT_DOC_FUND_AF: 144120 (64.65%)
ratio_MAT_DOC_INF_PRE: 127504 (57.19%)
ratio_MAT_DOC_FUND_AI: 120479 (54.04%)
ratio_MAT_DOC_INF: 115376 (51.75%)
ratio_MAT_DOC_FUND: 103654 (46.49%)
ratio_MAT_DOC_ESP_CC: 96437 (43.26%)
ratio_MAT_DOC_ESP: 94721 (42.49%)
QT_DOC_PROF: 52209 (23.42%)
QT_DOC_INF: 52209 (23.42%)
QT_DOC_INF_CRE: 52209 (23.42%)
QT_DOC_INF_PRE: 52209 (23.42%)
QT_DOC_FUND_AI: 52209 (23.42%)
QT_DOC_FUND_AF: 52209 (23.42%)
QT_DOC_MED: 52209 (23.42%)
QT_DOC_ESP_CE: 52209 (23.42%)
QT_DOC_PROF_TEC: 52209 (23.42%)
QT_DOC_EJA: 52209 (23.42%)
QT_DOC_EJA_FUND: 52209 (23.42%)
QT_DOC_EJA_MED: 52209 (23.42%)
QT_DOC_ESP: 52209 (23.42%)
QT_DOC_ES

/var/folders/2v/rfkdf8zd7yx_k7lff_2bchrr0000gn/T/ipykernel_8678/1781583595.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  f"{new_vars_na_count.index[i]}: {new_vars_na_count[i]} ({new_vars_na_prop[i]:.2%})"


In [20]:
# Now check missing values only in Para state
new_vars_na_count = (
    brazil_schools_census_data[brazil_schools_census_data["abbrev_state"] == "PA"][
        brazil_schools_census_data.columns[19:]
    ]
    .isna()
    .sum()
    .sort_values(ascending=False)
)
new_vars_na_prop = new_vars_na_count / len(
    brazil_schools_census_data[brazil_schools_census_data["abbrev_state"] == "PA"]
)

for i in range(0, len(new_vars_na_count)):
    print(
        f"{new_vars_na_count.index[i]}: {new_vars_na_count[i]} ({new_vars_na_prop[i]:.2%})"
    )

ratio_MAT_DOC_ESP_CE: 12575 (99.93%)
ratio_MAT_DOC_PROF_TEC: 12457 (98.99%)
ratio_MAT_DOC_PROF: 12434 (98.81%)
ratio_MAT_DOC_EJA_MED: 12262 (97.44%)
TP_CATEGORIA_ESCOLA_PRIVADA: 11765 (93.49%)
ratio_MAT_DOC_MED: 11726 (93.18%)
ratio_MAT_DOC_EJA_FUND: 10676 (84.84%)
ratio_MAT_DOC_EJA: 10461 (83.13%)
ratio_MAT_DOC_INF_CRE: 9605 (76.33%)
ratio_MAT_DOC_FUND_AF: 6020 (47.84%)
ratio_MAT_DOC_ESP_CC: 5947 (47.26%)
ratio_MAT_DOC_ESP: 5945 (47.24%)
ratio_MAT_DOC_INF_PRE: 5693 (45.24%)
ratio_MAT_DOC_INF: 5578 (44.33%)
ratio_MAT_DOC_FUND_AI: 4292 (34.11%)
ratio_MAT_DOC_FUND: 3693 (29.35%)
QT_DOC_INF: 2319 (18.43%)
QT_DOC_INF_CRE: 2319 (18.43%)
QT_DOC_INF_PRE: 2319 (18.43%)
QT_DOC_PROF: 2319 (18.43%)
QT_DOC_FUND_AI: 2319 (18.43%)
QT_DOC_FUND_AF: 2319 (18.43%)
QT_DOC_MED: 2319 (18.43%)
QT_DOC_ESP_CE: 2319 (18.43%)
QT_DOC_PROF_TEC: 2319 (18.43%)
QT_DOC_EJA: 2319 (18.43%)
QT_DOC_EJA_FUND: 2319 (18.43%)
QT_DOC_EJA_MED: 2319 (18.43%)
QT_DOC_ESP: 2319 (18.43%)
QT_DOC_ESP_CC: 2319 (18.43%)
ratio_MAT_DOC_B

/var/folders/2v/rfkdf8zd7yx_k7lff_2bchrr0000gn/T/ipykernel_8678/3529385462.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  f"{new_vars_na_count.index[i]}: {new_vars_na_count[i]} ({new_vars_na_prop[i]:.2%})"


In [21]:
# Now check missing values only in Florianopolis city
new_vars_na_count = (
    brazil_schools_census_data[
        brazil_schools_census_data["name_muni"] == "Florianópolis"
    ][brazil_schools_census_data.columns[19:]]
    .isna()
    .sum()
    .sort_values(ascending=False)
)
new_vars_na_prop = new_vars_na_count / len(
    brazil_schools_census_data[
        brazil_schools_census_data["name_muni"] == "Florianópolis"
    ]
)

for i in range(0, len(new_vars_na_count)):
    print(
        f"{new_vars_na_count.index[i]}: {new_vars_na_count[i]} ({new_vars_na_prop[i]:.2%})"
    )

ratio_MAT_DOC_ESP_CE: 351 (100.00%)
ratio_MAT_DOC_EJA_FUND: 346 (98.58%)
ratio_MAT_DOC_EJA_MED: 344 (98.01%)
ratio_MAT_DOC_EJA: 342 (97.44%)
ratio_MAT_DOC_PROF_TEC: 339 (96.58%)
ratio_MAT_DOC_PROF: 339 (96.58%)
ratio_MAT_DOC_MED: 291 (82.91%)
ratio_MAT_DOC_FUND_AF: 252 (71.79%)
ratio_MAT_DOC_FUND_AI: 228 (64.96%)
ratio_MAT_DOC_FUND: 223 (63.53%)
TP_CATEGORIA_ESCOLA_PRIVADA: 214 (60.97%)
ratio_MAT_DOC_INF_CRE: 184 (52.42%)
ratio_MAT_DOC_INF_PRE: 180 (51.28%)
ratio_MAT_DOC_INF: 171 (48.72%)
ratio_MAT_DOC_ESP: 114 (32.48%)
ratio_MAT_DOC_ESP_CC: 114 (32.48%)
QT_TUR_INF_CRE: 76 (21.65%)
QT_DOC_FUND_AF: 76 (21.65%)
QT_DOC_MED: 76 (21.65%)
QT_DOC_PROF: 76 (21.65%)
QT_DOC_PROF_TEC: 76 (21.65%)
QT_DOC_EJA: 76 (21.65%)
QT_DOC_EJA_FUND: 76 (21.65%)
QT_DOC_EJA_MED: 76 (21.65%)
QT_DOC_ESP: 76 (21.65%)
QT_DOC_ESP_CC: 76 (21.65%)
QT_DOC_ESP_CE: 76 (21.65%)
QT_TUR_BAS: 76 (21.65%)
QT_TUR_INF: 76 (21.65%)
QT_TUR_PROF_TEC: 76 (21.65%)
QT_TUR_INF_PRE: 76 (21.65%)
QT_TUR_FUND: 76 (21.65%)
QT_TUR_FUND_AI: 

/var/folders/2v/rfkdf8zd7yx_k7lff_2bchrr0000gn/T/ipykernel_8678/438510982.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  f"{new_vars_na_count.index[i]}: {new_vars_na_count[i]} ({new_vars_na_prop[i]:.2%})"


In [22]:
# Read indicators from INEP
ind_doc_esforco = pd.read_excel("data/IED_ESCOLAS_2022.xlsx", skiprows=10, skipfooter=5)

In [23]:
ind_doc_esforco.columns

Index(['NU_ANO_CENSO', 'NO_REGIAO', 'SG_UF', 'CO_MUNICIPIO', 'NO_MUNICIPIO',
       'CO_ENTIDADE', 'NO_ENTIDADE', 'NO_CATEGORIA', 'NO_DEPENDENCIA',
       'FUN_CAT_1', 'FUN_CAT_2', 'FUN_CAT_3', 'FUN_CAT_4', 'FUN_CAT_5',
       'FUN_CAT_6', 'FUN_AI_CAT_1', 'FUN_AI_CAT_2', 'FUN_AI_CAT_3',
       'FUN_AI_CAT_4', 'FUN_AI_CAT_5', 'FUN_AI_CAT_6', 'FUN_AF_CAT_1',
       'FUN_AF_CAT_2', 'FUN_AF_CAT_3', 'FUN_AF_CAT_4', 'FUN_AF_CAT_5',
       'FUN_AF_CAT_6', 'MED_CAT_1', 'MED_CAT_2', 'MED_CAT_3', 'MED_CAT_4',
       'MED_CAT_5', 'MED_CAT_6'],
      dtype='object')

In [24]:
ind_doc_esforco.head()

,NU_ANO_CENSO,NO_REGIAO,SG_UF,CO_MUNICIPIO,NO_MUNICIPIO,CO_ENTIDADE,NO_ENTIDADE,NO_CATEGORIA,NO_DEPENDENCIA,FUN_CAT_1,...,FUN_AF_CAT_3,FUN_AF_CAT_4,FUN_AF_CAT_5,FUN_AF_CAT_6,MED_CAT_1,MED_CAT_2,MED_CAT_3,MED_CAT_4,MED_CAT_5,MED_CAT_6
0,2022,Norte,RO,1100015,Alta Floresta D'Oeste,11022558,EIEEF HAP BITT TUPARI,Rural,Estadual,100,...,--,--,--,--,--,--,--,--,--,--
1,2022,Norte,RO,1100015,Alta Floresta D'Oeste,11024291,EMMEF 7 DE SETEMBRO,Rural,Municipal,100,...,0,0,0,0,--,--,--,--,--,--
2,2022,Norte,RO,1100015,Alta Floresta D'Oeste,11024372,EMEIEF ANA NERY,Urbana,Municipal,42.9,...,66.7,0,0,0,--,--,--,--,--,--
3,2022,Norte,RO,1100015,Alta Floresta D'Oeste,11024666,EMEIEF BOA ESPERANCA,Rural,Municipal,21,...,25,43.8,18.8,6.3,--,--,--,--,--,--
4,2022,Norte,RO,1100015,Alta Floresta D'Oeste,11024682,EEEFM EURIDICE LOPES PEDROSO,Urbana,Estadual,21,...,40,26.7,20,13.3,0,2.5,0,67.5,22.5,7.5


In [25]:
ind_doc_esforco.tail()

,NU_ANO_CENSO,NO_REGIAO,SG_UF,CO_MUNICIPIO,NO_MUNICIPIO,CO_ENTIDADE,NO_ENTIDADE,NO_CATEGORIA,NO_DEPENDENCIA,FUN_CAT_1,...,FUN_AF_CAT_3,FUN_AF_CAT_4,FUN_AF_CAT_5,FUN_AF_CAT_6,MED_CAT_1,MED_CAT_2,MED_CAT_3,MED_CAT_4,MED_CAT_5,MED_CAT_6
130902,2022,Centro-Oeste,DF,5300108,Brasília,53084020,ESC PEQUENO ENCANTO,Urbana,Privada,20,...,--,--,--,--,--,--,--,--,--,--
130903,2022,Centro-Oeste,DF,5300108,Brasília,53084039,COL ANCHIETA,Urbana,Privada,9.1,...,28.6,35.7,14.3,0,--,--,--,--,--,--
130904,2022,Centro-Oeste,DF,5300108,Brasília,53084055,COL MAPA,Urbana,Privada,29.4,...,54.5,36.4,0,0,0,7.6,38.5,30.8,15.4,7.7
130905,2022,Centro-Oeste,DF,5300108,Brasília,53085000,COBIAN - COL BIANGULO,Urbana,Privada,7.6,...,--,--,--,--,--,--,--,--,--,--
130906,2022,Centro-Oeste,DF,5300108,Brasília,53086007,CED VITORIA REGIA,Urbana,Privada,0,...,36.4,54.5,9.1,0,0,5.9,35.3,52.9,5.9,0


In [26]:
# Merge the Esforço Docente indicators with the main dataframe
br_schools_census_ide = brazil_schools_census_data.merge(
    ind_doc_esforco[["CO_ENTIDADE"] + ind_doc_esforco.columns[9:].tolist()],
    left_on="code_school",
    right_on="CO_ENTIDADE",
    how="left",
)

In [27]:
ind_doc_esforco_na_count = (
    br_schools_census_ide[ind_doc_esforco.columns[9:].tolist()].isna().sum()
)
ind_doc_esforco_na_prop = ind_doc_esforco_na_count / len(br_schools_census_ide)

print("Indicadores de esforço docente")
print(
    f"{ind_doc_esforco_na_count.unique()[0]} ({ind_doc_esforco_na_prop.unique()[0]:.2%})"
)

Indicadores de esforço docente
95413 (42.80%)


In [28]:
# Now check missing values only in Para state
ind_doc_esforco_na_count = (
    br_schools_census_ide[br_schools_census_ide["abbrev_state"] == "PA"][
        ind_doc_esforco.columns[9:].tolist()
    ]
    .isna()
    .sum()
)
ind_doc_esforco_na_prop = ind_doc_esforco_na_count / len(
    br_schools_census_ide[br_schools_census_ide["abbrev_state"] == "PA"]
)

print("Indicadores de esforço docente - PA")
print(
    f"{ind_doc_esforco_na_count.unique()[0]} ({ind_doc_esforco_na_prop.unique()[0]:.2%})"
)

Indicadores de esforço docente - PA
3295 (26.18%)


In [29]:
# Now check missing values only in Florianopolis city
ind_doc_esforco_na_count = (
    br_schools_census_ide[br_schools_census_ide["name_muni"] == "Florianópolis"][
        ind_doc_esforco.columns[9:].tolist()
    ]
    .isna()
    .sum()
)
ind_doc_esforco_na_prop = ind_doc_esforco_na_count / len(
    br_schools_census_ide[br_schools_census_ide["name_muni"] == "Florianópolis"]
)

print("Indicadores de esforço docente - Florianópolis")
print(
    f"{ind_doc_esforco_na_count.unique()[0]} ({ind_doc_esforco_na_prop.unique()[0]:.2%})"
)

Indicadores de esforço docente - Florianópolis
210 (59.83%)


In [30]:
ind_atu = pd.read_excel("data/ATU_ESCOLAS_2022.xlsx", skiprows=8, skipfooter=3)

In [31]:
ind_atu.head()

,NU_ANO_CENSO,NO_REGIAO,SG_UF,CO_MUNICIPIO,NO_MUNICIPIO,CO_ENTIDADE,NO_ENTIDADE,NO_CATEGORIA,NO_DEPENDENCIA,ED_INF_CAT_0,...,FUN_07_CAT_0,FUN_08_CAT_0,FUN_09_CAT_0,MULT_ETA_CAT_0,MED_CAT_0,MED_01_CAT_0,MED_02_CAT_0,MED_03_CAT_0,MED_04_CAT_0,MED_NS_CAT_0
0,2022,Norte,RO,1100015,Alta Floresta D'Oeste,11022558,EIEEF HAP BITT TUPARI,Rural,Estadual,--,...,--,--,--,--,--,--,--,--,--,--
1,2022,Norte,RO,1100015,Alta Floresta D'Oeste,11024275,CEEJA LUIZ VAZ DE CAMOES,Urbana,Estadual,--,...,--,--,--,--,--,--,--,--,--,--
2,2022,Norte,RO,1100015,Alta Floresta D'Oeste,11024291,EMMEF 7 DE SETEMBRO,Rural,Municipal,--,...,--,--,--,9,--,--,--,--,--,--
3,2022,Norte,RO,1100015,Alta Floresta D'Oeste,11024372,EMEIEF ANA NERY,Urbana,Municipal,17,...,14,11,16,14,--,--,--,--,--,--
4,2022,Norte,RO,1100015,Alta Floresta D'Oeste,11024666,EMEIEF BOA ESPERANCA,Rural,Municipal,18,...,17,14,26,18,--,--,--,--,--,--


In [32]:
ind_atu.tail()

,NU_ANO_CENSO,NO_REGIAO,SG_UF,CO_MUNICIPIO,NO_MUNICIPIO,CO_ENTIDADE,NO_ENTIDADE,NO_CATEGORIA,NO_DEPENDENCIA,ED_INF_CAT_0,...,FUN_07_CAT_0,FUN_08_CAT_0,FUN_09_CAT_0,MULT_ETA_CAT_0,MED_CAT_0,MED_01_CAT_0,MED_02_CAT_0,MED_03_CAT_0,MED_04_CAT_0,MED_NS_CAT_0
174314,2022,Centro-Oeste,DF,5300108,Brasília,53084020,ESC PEQUENO ENCANTO,Urbana,Privada,14,...,--,--,--,--,--,--,--,--,--,--
174315,2022,Centro-Oeste,DF,5300108,Brasília,53084039,COL ANCHIETA,Urbana,Privada,14.9,...,28,22,20,--,--,--,--,--,--,--
174316,2022,Centro-Oeste,DF,5300108,Brasília,53084055,COL MAPA,Urbana,Privada,12,...,28,42,29,--,31,28,34,31,--,--
174317,2022,Centro-Oeste,DF,5300108,Brasília,53085000,COBIAN - COL BIANGULO,Urbana,Privada,17.3,...,--,--,--,--,--,--,--,--,--,--
174318,2022,Centro-Oeste,DF,5300108,Brasília,53086007,CED VITORIA REGIA,Urbana,Privada,--,...,--,--,19,--,24.6,27,22,26,--,--


In [33]:
# Merge the indicators of teacher effort and teacher qualification
br_schools_census_ide_atu = br_schools_census_ide.merge(
    ind_atu[["CO_ENTIDADE"] + ind_atu.columns[9:].tolist()],
    left_on="code_school",
    right_on="CO_ENTIDADE",
    how="left",
)

In [34]:
ind_atu_na_count = br_schools_census_ide_atu[ind_atu.columns[9:].tolist()].isna().sum()
ind_atu_na_prop = ind_atu_na_count / len(br_schools_census_ide_atu)

print("Indicador de Média de Alunos por Turma")
print(f"{ind_atu_na_count.unique()[0]} ({ind_atu_na_prop.unique()[0]:.2%})")

Indicador de Média de Alunos por Turma
55699 (24.98%)


In [35]:
# Now check missing values only in Para state
ind_atu_na_count = (
    br_schools_census_ide_atu[br_schools_census_ide_atu["abbrev_state"] == "PA"][
        ind_atu.columns[9:].tolist()
    ]
    .isna()
    .sum()
)
ind_atu_na_prop = ind_atu_na_count / len(
    br_schools_census_ide_atu[br_schools_census_ide_atu["abbrev_state"] == "PA"]
)

print("Indicador de Média de Alunos por Turma - PA")
print(f"{ind_atu_na_count.unique()[0]} ({ind_atu_na_prop.unique()[0]:.2%})")

Indicador de Média de Alunos por Turma - PA
2367 (18.81%)


In [36]:
# Now check missing values only in Florianopolis city
ind_atu_na_count = (
    br_schools_census_ide_atu[
        br_schools_census_ide_atu["name_muni"] == "Florianópolis"
    ][ind_atu.columns[9:].tolist()]
    .isna()
    .sum()
)
ind_atu_na_prop = ind_atu_na_count / len(
    br_schools_census_ide_atu[br_schools_census_ide_atu["name_muni"] == "Florianópolis"]
)

print("Indicador de Média de Alunos por Turma - Florianópolis")
print(f"{ind_atu_na_count.unique()[0]} ({ind_atu_na_prop.unique()[0]:.2%})")

Indicador de Média de Alunos por Turma - Florianópolis
84 (23.93%)


In [37]:
# Convert "--" to NaN and convert columns to float for Esforço Docente indicator
br_schools_census_ide_atu.loc[:, ind_doc_esforco.columns[9:].tolist()] = (
    br_schools_census_ide_atu[ind_doc_esforco.columns[9:].tolist()]
    .replace("--", np.nan)
    .astype(float)
)

# Convert "--" to NaN and convert columns to float for Média de Alunos por Turma indicator
br_schools_census_ide_atu.loc[:, ind_atu.columns[9:].tolist()] = (
    br_schools_census_ide_atu[ind_atu.columns[9:].tolist()]
    .replace("--", np.nan)
    .astype(float)
)

In [38]:
# Create new columns for latitude and longitude
br_schools_census_ide_atu["lat"] = br_schools_census_ide_atu.geometry.y
br_schools_census_ide_atu["lon"] = br_schools_census_ide_atu.geometry.x

In [39]:
# Save data as CSV without geometry column
br_schools_census_ide_atu.drop("geometry", axis=1).to_csv(
    "data/br_schools_census_ide_atu.csv", index=False
)

In [40]:
# Save data as Parquet with geometry column
br_schools_census_ide_atu.to_parquet(
    "data/brazil_schools_census_education_metrics.parquet"
)

In [41]:
# Save only for the Para state data as CSV
br_schools_census_ide_atu[br_schools_census_ide_atu["abbrev_state"] == "PA"].to_parquet(
    "data/para_schools_census_education_metrics.parquet"
)